In [26]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim
import math
import numpy as np
import cv2
from PIL import Image
from torch.utils.data import Dataset, DataLoader, random_split
from CustomDataset import CustomDataset




In [27]:
#Initializations
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = torchvision.models.segmentation.fcn_resnet50(pretrained=True).to(device)

#has to normalize data the same way the pretrained images were
normalize = transforms.Normalize(mean=[0.485,0.456,0.406],
                        std=[0.229,0.224,0.225])
data_transforms = {
    'train':
    transforms.Compose([
        transforms.ToTensor(),
        normalize
    ]),
    'validation':
    transforms.Compose([
        transforms.ToTensor(),
        normalize
    ]),
}
dataset = CustomDataset(target_type = 'poly')

TRAIN_SIZE= math.floor(dataset.__len__()*0.75)
TEST_SIZE = dataset.__len__() - TRAIN_SIZE
trainset, testset = random_split(dataset,[TRAIN_SIZE,TEST_SIZE])

traindata_loader = DataLoader(dataset, batch_size=10, shuffle=True)
testdata_loader = DataLoader(dataset, batch_size=10, shuffle=True)
# data_loader = DataLoader(dataset, batch_size=10, shuffle=True)
image_datasets = {
    'train': 
        TRAIN_SIZE,
    'validation':
        TEST_SIZE
}

dataloaders = {
    'train':
        traindata_loader,
    'validation':
        testdata_loader
}

for param in model.parameters():
    param.requires_grad = False   
    
model.fc = nn.Sequential(
               nn.Linear(2048, 128),
               nn.ReLU(inplace=True),
               nn.Linear(128, 2)).to(device)

loss_fn = nn.CrossEntropyLoss()
optim = optim.Adam((model.fc.parameters()))

In [29]:
for batch, (inputs,labels) in enumerate(dataloaders['train']):
# for inputs,labels in enumerate(dataloaders[phase]):
    # inputs = inputs.to(device)
    # labels = labels.to(device)
    print(inputs.shape)
    print(labels.dtype)
    break

torch.Size([10, 3, 960, 1280])
torch.float32


In [30]:
def train_model(model, criterion, optimizer, num_epochs=3):
    for epoch in range(num_epochs):
        for phase in ['train', 'validation']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0
            for batch, (inputs,labels) in enumerate(dataloaders[phase]):
            # for inputs,labels in enumerate(dataloaders[phase]):
                inputs = inputs.to(device)
                labels = labels.to(device)
                # print(inputs.dtype)
                # print(labels.dtype)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                if phase == 'train':
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()
                _, preds = torch.max(outputs, 1)
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / image_datasets[phase]
            epoch_acc = running_corrects.double() / image_datasets[phase]

            print('{} loss: {:.4f}, acc: {:.4f}'.format(phase,
                                                        epoch_loss,
                                                        epoch_acc))
    return model


In [31]:
model_trained = train_model(model, loss_fn, optim, num_epochs=3)
!mkdir models
!mkdir models/pytorch
torch.save(model_trained.state_dict(), 'models/pytorch/weights.h5')





torch.float32
torch.float32


KeyboardInterrupt: 

In [ ]:
model = torchvision.models.segmentation.fcn_resnet50(pretrained=False).to(device)
model.fc = nn.Sequential(
               nn.Linear(2048, 128),
               nn.ReLU(inplace=True),
               nn.Linear(128, 2)).to(device)
model.load_state_dict(torch.load('models/pytorch/weights.h5'))